In [58]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

# Pkg.add("CoordinateTransformations")
# Pkg.add("RigidBodyDynamics")
# Pkg.add("MeshCatMechanisms")
# Pkg.add("MeshCat")

using CoordinateTransformations: Translation
using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using Printf
using LinearAlgebra

  Activating project at `~/cmu/courses/16745_OCRL/zoe2_optimal_controller/julia`


In [59]:
vis = Visualizer()
# open(vis)  # open the visualizer in a separate tab/window
render(vis) # render the visualizer here inside the jupyter notebook

┌ Info: Listening on: 127.0.0.1:8745, thread id: 1
└ @ HTTP.Servers /home/hayden/.julia/packages/HTTP/4AUPl/src/Servers.jl:382
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8745
└ @ MeshCat /home/hayden/.julia/packages/MeshCat/9QrxD/src/visualizer.jl:43


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{HTTP.WebSockets.WebSocket}(), ip"127.0.0.1", 8745, HTTP.Servers.Server{HTTP.Servers.Listener{Nothing, Sockets.TCPServer}}(HTTP.Servers.Listener{Nothing, Sockets.TCPServer}(Sockets.InetAddr{Sockets.IPv4}(ip"127.0.0.1", 8745), "127.0.0.1", "8745", nothing, Sockets.TCPServer(RawFD(120) active)), nothing, Set{HTTP.Connections.Connection}(), Task (runnable) @0x00007154b56cabd0, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (707, 7811903076430078976, 6))), MeshCat.AnimationContext[]))

In [60]:
urdf = joinpath(dirname(pathof(MeshCatMechanisms)), "..", "test", "urdf", "Acrobot.urdf")
robot = parse_urdf(urdf)
delete!(vis)
mvis = MechanismVisualizer(robot, URDFVisuals(urdf), vis)
set_configuration!(mvis, [0.0, 0.0])

In [61]:
state = MechanismState(robot, randn(2), randn(2))
t, q, v = simulate(state, 5.0);

## Animating a Trajectory

You can animate a trajectory (a list of times and configurations) by creating an `Animation` and calling `setanimation!` to pass it to the visuailzer:

In [62]:
animation = Animation(mvis, t, q)
setanimation!(mvis, animation)

### Replaying the Animation

When you use `setanimation!`, the entire animation is sent to the visualizer, so you can replay it at will. In the viewer, click "Open Controls" in the top right corner, go to "Animation" -> "default" and click "play" to play the animation again. 

### Recording the Animation

You can also record an animation directly from the MeshCat viewer. Just open the controls and go to "Animation" -> "default" -> "Recording" and click "record". The animation will play back (slowly, as it needs to cleanly capture every frame), and then you'll be able to download the resulting frames. 

Currently, MeshCat downloads an animation as a `.tar` file containing all of the individual frames. To convert that into a video, you just need to install the `ffmpeg` program (`apt-get install ffmpeg` on Ubuntu 18.04) and then run: 

```julia
MeshCat.convert_frames_to_video("/path/to/downloads/meshcat_xxx.tar")
```

### Visualizing Frames and Points

You can use the lower-level `setelement!()` function to attach geometries to frames in the mechanism. For example, to visualize a coordinate frame with a triad, you can do: 

In [63]:
lower_arm = bodies(robot)[end]
body_frame = default_frame(lower_arm)
setelement!(mvis, body_frame)

MeshCat Visualizer with path /meshcat/world/upper_link/lower_link/after_elbow/<element> at http://127.0.0.1:8745

And likewise, to visualize a point attached to a frame, you can do:

In [64]:
radius = 0.05
name = "my_point"
setelement!(mvis, Point3D(body_frame, 0.2, 0.2, 0.2), radius, name)

MeshCat Visualizer with path /meshcat/world/upper_link/lower_link/after_elbow/my_point at http://127.0.0.1:8745

The triad and point are included in the MeshCat scene tree, so each will move along with the body to which it's attached. 


Try clicking "Animation" -> "default" -> "play" again in the MeshCat window. You should see the arm animation repeat, and the point and triad you've added will move as well. 

Now let's try loading a more complex robot like NASA's Valkyrie (a humanoid robot).

In [65]:
Pkg.activate(temp=true)
Pkg.add("ValkyrieRobot")
using ValkyrieRobot

val = Valkyrie();
delete!(vis)
urdfvisuals = URDFVisuals(
    ValkyrieRobot.urdfpath(),
    package_path=[dirname(dirname(ValkyrieRobot.urdfpath()))])
mvis = MechanismVisualizer(val.mechanism, urdfvisuals, vis);

  Activating new project at `/tmp/jl_BaMssE`
   Resolving package versions...
    Updating `/tmp/jl_BaMssE/Project.toml`
  [c74b26e8] + ValkyrieRobot v0.2.1
    Updating `/tmp/jl_BaMssE/Manifest.toml`
  [ffbed154] + DocStringExtensions v0.9.4
  [692b3bcd] + JLLWrappers v1.7.0
  [9c8b4983] + LightXML v0.9.1
  [39f5be34] + LoopThrottle v0.1.0
⌅ [aea7be01] + PrecompileTools v1.2.1
  [21216c6a] + Preferences v1.4.3
  [94ee1d12] + Quaternions v0.7.6
  [c1ae055f] + RealDot v0.1.0
  [189a3867] + Reexport v1.2.2
  [366cf18f] + RigidBodyDynamics v2.5.0
  [6038ab10] + Rotations v1.7.1
  [90137ffa] + StaticArrays v1.9.13
  [1e83bf80] + StaticArraysCore v1.4.3
  [94a5cd58] + TypeSortedCollections v1.1.0
  [c4a57d5a] + UnsafeArrays v1.0.7
  [c74b26e8] + ValkyrieRobot v0.2.1
  [94ce4f54] + Libiconv_jll v1.18.0+0
  [02c8fc9c] + XML2_jll v2.13.6+1
  [56f22d72] + Artifacts
  [2a0f44e3] + Base64
  [ade2ca70] + Dates
  [b77e0a4c] + InteractiveUtils
  [8f399da3] + Libdl
  [37e2e46d] + LinearAlgebra
  [56d

Let's change the configuration from all zeros to a "nominal" configuration.

In [66]:
using ValkyrieRobot.BipedControlUtil: Side, flipsign_if_right

function initialize!(state::MechanismState, val::Valkyrie)
    zero!(state)
    mechanism = val.mechanism
    for side in instances(Side)
        set_configuration!(state, findjoint(mechanism, "$(side)KneePitch"), [1.205])
        set_configuration!(state, findjoint(mechanism, "$(side)HipPitch"), [-0.49])
        set_configuration!(state, findjoint(mechanism, "$(side)AnklePitch"), [-0.71])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderPitch"), [0.300196631343025])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderRoll"), [flipsign_if_right(-1.25, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ElbowPitch"), [flipsign_if_right(-0.785398163397448, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ForearmYaw"), [1.571])
    end
    set_configuration!(state, val.basejoint, [1; 0; 0; 0; 0; 0; 1.025])
    nothing
end

state = MechanismState(val.mechanism)
initialize!(state, val)

set_configuration!(mvis, configuration(state))

We can also visualise the inertias of the robot's individual linkages as a set of ellipsoids.

The snippet below adds this visual representation to the 3D viewer, side-by-side with the actual robot.

In [67]:
mvis = MechanismVisualizer(val.mechanism, Skeleton(), vis[:val_inertia])
set_configuration!(mvis, configuration(state))
settransform!(vis[:val_inertia], Translation(0, 2, 0))

MeshCat Visualizer with path /meshcat/val_inertia at http://127.0.0.1:8745

---

In [68]:
# Handling robots with fixed joints
urdf = joinpath(dirname(pathof(MeshCatMechanisms)), "..", "test", "urdf", "Acrobot.urdf")
robot = parse_urdf(urdf)
RigidBodyDynamics.remove_fixed_tree_joints!(robot)
vis = MechanismVisualizer(robot, URDFVisuals(urdf))

# with the elbow fixed, there is only one configuration element
set_configuration!(vis, [0.5]) 
render(vis)

┌ Info: Listening on: 127.0.0.1:8746, thread id: 1
└ @ HTTP.Servers /home/hayden/.julia/packages/HTTP/4AUPl/src/Servers.jl:382
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8746
└ @ MeshCat /home/hayden/.julia/packages/MeshCat/9QrxD/src/visualizer.jl:43


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "upper_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x75, 0x70, 0x70, 0x65, 0x72, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("lower_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6c, 0x6f, 0x77, 0x65, 0x72, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_elbow" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x74, 0x65, 0x72, 0x5f, 0x65, 0x6c, 0x62, 0x6f, 0x77], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), "after_shoulder" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x5f, 0x73, 0x68, 0x6f, 0x75, 0x6c, 0x64, 0x65, 0x72], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))))))), Set{HTTP.WebSockets.WebSocket}(), ip"127.0.0.1", 8746, HTTP.Servers.Server{HTTP.Servers.Listener{Nothing, Sockets.TCPServer}}(HTTP.Servers.Listener{Nothing, Sockets.TCPServer}(Sockets.InetAddr{Sockets.IPv4}(ip"127.0.0.1", 8746), "127.0.0.1", "8746", nothing, Sockets.TCPServer(RawFD(124) active)), nothing, Set{HTTP.Connections.Connection}(), Task (runnable) @0x00007154c85abd00, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (41, 124609171891456, 9223372036854251520))), MeshCat.AnimationContext[]))